In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df_projects = pd.read_csv('Limited Grants Dataset - ExPORTER.CSV', header=0)
df_link = pd.read_csv('Limited Publication Project Linkage Dataset - ExPORTER.CSV', header=0)
df_pub = pd.read_csv('Limited Publications Dataset - ExPORTER.CSV', header=0, encoding='iso-8859-1')

In [3]:
# clean data
before_2001 = df_projects['FY'] < 2001
after_2015  = df_projects['FY'] > 2015
df_projects = df_projects[(~before_2001) & (~after_2015)]
df_temp = df_projects[['PI_IDS','PI_NAMEs','CORE_PROJECT_NUM', 'FY', 'IC_NAME' ]]
df_temp.IC_NAME.where(df_temp.IC_NAME == 'NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES', 'others', inplace=True)
df_temp.IC_NAME.where(df_temp.IC_NAME == 'others', 'nigms', inplace=True)
df_temp.IC_NAME.unique()

/Users/weizheng/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


array(['others', 'nigms'], dtype=object)

In [4]:
# looping every row of the dataset to create  equal length of arrays for pid, p_names,
# project number, fiscal years, and ic
vals = df_temp.values;
rows, cols = vals.shape
p_ids = []
p_names = []
p_nums = []
fys = []
ics = []
#grab each pid from the ';' separated strings
for i in range(0,rows):
    if vals[i, 0] is not None and vals[i, 1] is not None:
        ids = vals[i, 0].strip().split(';')
        names = vals[i,1].strip().split(';')
        ids.pop()
        names.pop()
        if len(ids) == len(names):
            #if there are more than one pid, take primary contact
            if len(ids) == 1:
                ids = [d.strip() for d in ids ]
                names = [n.strip() for n in names]
            else:
                #ids = [d.replace(' (contact)', '').strip() for d in ids if '(contact)'   in d]
                #names = [n.replace(' (contact)', '').strip() for n in names if '(contact)' in n]
                ids = [d.replace(' (contact)', '').strip() for d in ids ]
                names = [n.replace(' (contact)', '').strip() for n in names]
            p_num = vals[i,2]
            fy = vals[i,3]
            ic = vals[i,4]
            p_ids.extend(ids)
            p_names.extend(names)
            p_nums.extend([str(p_num)] * len(ids))
            fys.extend([fy] * len(ids))
            ics.extend([ic] * len(ids))

In [5]:
# create unique combinations of the above arrays
def getUniqDF(A1, A2):
    #taking two equal-lenght array, create a datafram of unique pairs
    a1_a2s = [str(a1) + '_' + str(a2) for a1, a2 in zip(A1, A2)]
    uniq_pair = set(a1_a2s)
    uniq_1 = []
    uniq_2 = []
    #unique name and pid combinations
    for a1_a2 in uniq_pair:
        s = a1_a2.split('_')
        uniq_1.append(s[0])
        uniq_2.append(s[1])
    return(uniq_1, uniq_2)
# pid and names
uniq_pids, uniq_names = getUniqDF(p_ids, p_names)
df_pid_name = pd.DataFrame({'pid': uniq_pids, 'pname':uniq_names})

# pids and project number
pid4num, uniq_prjnum = getUniqDF(p_ids, p_nums)
df_pid_prjnum = pd.DataFrame({'pid': pid4num, 'prjnum':uniq_prjnum})

# pids, fy and ic
combs = [str(a1) + '_' + str(a2) + '_' + str(a3) for a1, a2, a3 in zip(p_ids, fys, ics)]
uniq_combinations = set(combs)
uniq_1 = []
uniq_2 = []
uniq_3 = []

for a1_a2_a3 in uniq_combinations:
    s = a1_a2_a3.split('_')
    uniq_1.append(s[0])
    uniq_2.append(s[1])
    uniq_3.append(s[2])
    
df_pid_fy_ic = pd.DataFrame({'pid': uniq_1, 'fy':uniq_2, 'ic': uniq_3 })

In [6]:
#calculate years of support 
df_pid_fy_ic['val'] = 1
df_sum = df_pid_fy_ic[['pid', 'ic', 'val']]
df_support = df_sum.pivot_table(columns='ic', index='pid', values='val', aggfunc='sum')
df_support.columns = ['nigms_years', 'others_years']
df_support['nih_years'] =  df_support[['nigms_years', 'others_years']].apply(np.max, axis=1)


### Question 1
Identify the number of unique researchers connected to these grants


In [7]:
df_support.index

Index(['10124285', '10531840', '11334526', '1857690', '1858254', '1858379',
       '1859097', '1861156', '1861604', '1862083', '1862952', '1865319',
       '1865665', '1867112', '1868336', '1868920', '1872778', '1872999',
       '1876102', '1877980', '1878565', '1880797', '1883114', '1883128',
       '1884183', '1884252', '1885109', '1885775', '1886752', '1887701',
       '1888492', '1889248', '1889808', '1893745', '1898338', '1898413',
       '1901683', '1901765', '1922021', '1942514', '1950535', '1964213',
       '1964483', '1964871', '1973848', '2094386', '2210723', '2212695',
       '2403651', '3063608', '3130466', '6599082', '6799828', '7733546',
       '7845920', '8032961', '8122521', '8436442', '8678128', '8959346',
       '9201630', '9316806', '9349122'],
      dtype='object', name='pid')

In [8]:
len(df_support.index)

63

### NIGMS Researchers

In [9]:
df_support[df_support.nigms_years>1].index

Index(['10531840', '1857690', '1858254', '1858379', '1859097', '1861156',
       '1861604', '1862083', '1865319', '1867112', '1872999', '1876102',
       '1878565', '1883128', '1885109', '1887701', '1889808', '1898413',
       '1901683', '1964483', '1964871', '7733546'],
      dtype='object', name='pid')

In [10]:
len(df_support[df_support.nigms_years>1].index)



22

#### Researchers who were continually supported by NIMGS

In [13]:
df_support[df_support.nigms_years==15] 

,nigms_years,others_years,nih_years
pid,,,
1858379,15.0,NaN,15.0
1865319,15.0,11.0,15.0
1883128,15.0,9.0,15.0
1885109,15.0,6.0,15.0
1887701,15.0,8.0,15.0
1889808,15.0,6.0,15.0
1898413,15.0,7.0,15.0
1901683,15.0,14.0,15.0
1964483,15.0,3.0,15.0


#### Researchers who were continually supported by NIH

In [15]:
df_support[(df_support.nih_years==15) & (df_support.nigms_years > 1)]

,nigms_years,others_years,nih_years
pid,,,
1858254,5.0,15.0,15.0
1858379,15.0,NaN,15.0
1861604,7.0,15.0,15.0
1865319,15.0,11.0,15.0
1867112,8.0,15.0,15.0
1872999,10.0,15.0,15.0
1883128,15.0,9.0,15.0
1885109,15.0,6.0,15.0
1887701,15.0,8.0,15.0


### Question 2
For these investigators, identify the number of years of NIH support and NIGMS support (a subset of NIH support) that they have had for these grants

In [439]:
df_support

,nigms_years,others_years,nih_years
pid,,,
10531840,4.0,NaN,4.0
11334526,NaN,1.0,1.0
1857690,14.0,13.0,14.0
1858254,5.0,15.0,15.0
1858379,15.0,NaN,15.0
1859097,13.0,13.0,13.0
1861156,11.0,13.0,13.0
1861604,7.0,13.0,13.0
1862083,11.0,13.0,13.0


### Question 3
identify how many publications on which each of these researchers has been an author

In [440]:
df_support = df_support.reset_index()
#nigms researchers
df_nigms_pid = pd.DataFrame(df_support.pid[df_support.nigms_years>0])
#look up projection information
df_nigms_pid_pnum = df_nigms_pid.merge(df_pid_prjnum, left_on='pid', right_on='pid', how='inner')
#look up publication id
df_nigms_link = df_nigms_pid_pnum.merge(df_link, left_on='prjnum', right_on='PROJECT_NUMBER', how='inner')
#look up publications
df_nigms_pid_pub = df_nigms_link.merge(df_pub, left_on='PMID', right_on='PMID', how='inner')

####  NIGMS Researchers' publications

In [441]:
df_nigms_pid_pub[['PMID','pid']].groupby('pid').count()

,PMID
pid,
10531840,25
1857690,350
1858254,401
1858379,35
1859097,241
1861156,70
1861604,984
1862083,796
1862952,256


In [442]:
df_nigms_pid_pub

,pid,prjnum,PMID,PROJECT_NUMBER,AFFILIATION,AUTHOR_LIST,COUNTRY,ISSN,JOURNAL_ISSUE,JOURNAL_TITLE,JOURNAL_TITLE_ABBR,JOURNAL_VOLUME,LANG,PAGE_NUMBER,PMC_ID,PUB_DATE,PUB_TITLE,PUB_YEAR
0,10531840,R01GM098861,22223736,R01GM098861,"Division of Basic Sciences, Fred Hutchinson Ca...","Mak, Amanda Nga-Sze; Bradley, Philip; Cernadas...",United States,1095-9203,6069,"Science (New York, N.Y.)",Science,335,ENG,716-9,3427646.0,2012 Feb 10,The crystal structure of TAL effector PthXo1 b...,2012
1,1859097,R01GM049857,22223736,R01GM049857,"Division of Basic Sciences, Fred Hutchinson Ca...","Mak, Amanda Nga-Sze; Bradley, Philip; Cernadas...",United States,1095-9203,6069,"Science (New York, N.Y.)",Science,335,ENG,716-9,3427646.0,2012 Feb 10,The crystal structure of TAL effector PthXo1 b...,2012
2,10531840,R01GM098861,22916025,R01GM098861,"Department of Human Genetics, University of Ut...","Dahlem, Timothy J; Hoshijima, Kazuyuki; Juryne...",United States,1553-7404,8,PLoS genetics,PLoS Genet,8,eng,e1002861,3420959.0,2012,Simple methods for generating and detecting lo...,2012
3,10531840,R01GM098861,22693217,R01GM098861,Department of Plant Pathology and Microbiology...,"Doyle, Erin L; Booher, Nicholas J; Standage, D...",England,1362-4962,Web Server issue,Nucleic acids research,Nucleic Acids Res,40,eng,W117-22,3394250.0,2012 Jul,TAL Effector-Nucleotide Targeter (TALE-NT) 2.0...,2012
4,10531840,R01GM098861,23028976,R01GM098861,"Department of Genetics, Cell Biology & Develop...","Christian, Michelle L; Demorest, Zachary L; St...",United States,1932-6203,9,PloS one,PLoS One,7,eng,e45383,3454392.0,2012,Targeting G with TAL effectors: a comparison o...,2012
5,10531840,R01GM098861,23707478,R01GM098861,Department of Plant Pathology and Microbiology...,"Doyle, Erin L; Stoddard, Barry L; Voytas, Dani...",England,1879-3088,8,Trends in cell biology,Trends Cell Biol,23,eng,390-8,3729746.0,2013 Aug,TAL effectors: highly adaptable phytobacterial...,2013
6,10531840,R01GM098861,23288864,R01GM098861,NaN,"Shan, Qiwei; Wang, Yanpeng; Chen, Kunling; Lia...",England,1752-9867,4,Molecular plant,Mol Plant,6,eng,1365-8,3968307.0,2013 Jul,Rapid and efficient gene modification in rice ...,2013
7,10531840,R01GM098861,24101480,R01GM098861,"Masonic Cancer Center, Graduate Program in Mol...","Nyquist, Michael D; Li, Yingming; Hwang, Tae H...",United States,1091-6490,43,Proceedings of the National Academy of Science...,Proc Natl Acad Sci U S A,110,eng,17492-7,3808622.0,2013 Oct 22,TALEN-engineered AR gene rearrangements reveal...,2013
8,10531840,R01GM098861,23979944,R01GM098861,"Department of Genetics, Cell Biology, and Deve...","Christian, Michelle; Qi, Yiping; Zhang, Yong; ...",United States,2160-1836,10,"G3 (Bethesda, Md.)",G3 (Bethesda),3,eng,1697-705,3789794.0,2013 Oct 03,Targeted mutagenesis of Arabidopsis thaliana u...,2013
9,10531840,R01GM098861,23689819,R01GM098861,"Research Centre Flakkebjerg, Department of Mol...","Wendt, Toni; Holm, Preben Bach; Starker, Colby...",Netherlands,1573-5028,3,Plant molecular biology,Plant Mol Biol,83,eng,279-85,NaN,2013 Oct,TAL effector nucleases induce mutations at a p...,2013
